In [ ]:
#Para acessar os arquivos do Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
import numpy as np

In [ ]:
# Caminhos no Drive
path = '/content/drive/MyDrive/UFC/TCC/odonto/history/metrics_new'
figures_path = '/content/drive/MyDrive/UFC/TCC/odonto/curvas_aprendizado'
os.makedirs(figures_path, exist_ok=True)

In [ ]:
# Lista dos nomes dos filtros e modelos utilizados
preprocessamentos = ['images', 'subtract_laplacian', 'subtract_sobel', 'subtract_prewitt',
                     'subtract_roberts', 'subtract_blackhat', 'gammaCorrection', 'add_tophat']
modelos = ['InceptionV3', 'EfficientNetV2B0', 'VGG19']

In [ ]:
# Função para plotar e salvar curvas de aprendizado
def plot_learning_curves(preproc, model):
    pattern = f'{path}/{preproc}_{model}_fold*.csv'
    files = glob.glob(pattern)

    if not files:
        print(f"⚠️ Nenhum histórico encontrado para {preproc} + {model}")
        return

    all_histories = []

    for file in files:
        df = pd.read_csv(file)
        all_histories.append(df)

    # Garantir alinhamento pelo menor número de épocas
    min_epochs = min([len(h) for h in all_histories])
    all_histories = [h.iloc[:min_epochs] for h in all_histories]

    # Empilhar os históricos e calcular média/STD
    stacked = np.stack([h[['accuracy', 'val_accuracy', 'loss', 'val_loss']].values for h in all_histories])
    mean_vals = np.mean(stacked, axis=0)
    std_vals = np.std(stacked, axis=0)
    epochs = np.arange(min_epochs)

    # Criar gráfico
    plt.figure(figsize=(12, 6))

    # Acurácia
    plt.subplot(1, 2, 1)
    plt.plot(epochs, mean_vals[:, 0], label='Treino')
    plt.fill_between(epochs, mean_vals[:, 0] - std_vals[:, 0], mean_vals[:, 0] + std_vals[:, 0], alpha=0.2)
    plt.plot(epochs, mean_vals[:, 1], label='Validação')
    plt.fill_between(epochs, mean_vals[:, 1] - std_vals[:, 1], mean_vals[:, 1] + std_vals[:, 1], alpha=0.2)
    plt.title(f'Acurácia - {preproc} + {model}')
    plt.xlabel('Época')
    plt.ylabel('Acurácia')
    plt.legend()

    # Loss
    plt.subplot(1, 2, 2)
    plt.plot(epochs, mean_vals[:, 2], label='Treino')
    plt.fill_between(epochs, mean_vals[:, 2] - std_vals[:, 2], mean_vals[:, 2] + std_vals[:, 2], alpha=0.2)
    plt.plot(epochs, mean_vals[:, 3], label='Validação')
    plt.fill_between(epochs, mean_vals[:, 3] - std_vals[:, 3], mean_vals[:, 3] + std_vals[:, 3], alpha=0.2)
    plt.title(f'Loss - {preproc} + {model}')
    plt.xlabel('Época')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()

    # Salvar figura
    filename = f'curva_{preproc}_{model}.png'
    save_path = os.path.join(figures_path, filename)
    plt.savefig(save_path, dpi=300)
    plt.close()
    print(f'✅ Gráfico salvo: {save_path}')

# Gerar e salvar gráficos para todos os modelos e filtros
for model in modelos:
    for preproc in preprocessamentos:
        plot_learning_curves(preproc, model)

✅ Gráfico salvo: /content/drive/MyDrive/UFC/TCC/odonto/curvas_aprendizado/curva_images_InceptionV3.png
✅ Gráfico salvo: /content/drive/MyDrive/UFC/TCC/odonto/curvas_aprendizado/curva_subtract_laplacian_InceptionV3.png
✅ Gráfico salvo: /content/drive/MyDrive/UFC/TCC/odonto/curvas_aprendizado/curva_subtract_sobel_InceptionV3.png
✅ Gráfico salvo: /content/drive/MyDrive/UFC/TCC/odonto/curvas_aprendizado/curva_subtract_prewitt_InceptionV3.png
✅ Gráfico salvo: /content/drive/MyDrive/UFC/TCC/odonto/curvas_aprendizado/curva_subtract_roberts_InceptionV3.png
✅ Gráfico salvo: /content/drive/MyDrive/UFC/TCC/odonto/curvas_aprendizado/curva_subtract_blackhat_InceptionV3.png
✅ Gráfico salvo: /content/drive/MyDrive/UFC/TCC/odonto/curvas_aprendizado/curva_gammaCorrection_InceptionV3.png
✅ Gráfico salvo: /content/drive/MyDrive/UFC/TCC/odonto/curvas_aprendizado/curva_add_tophat_InceptionV3.png
✅ Gráfico salvo: /content/drive/MyDrive/UFC/TCC/odonto/curvas_aprendizado/curva_images_EfficientNetV2B0.png
✅ Gr